In [9]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import json

CHROMA_DIR = "/data/edutem/sooine/rag_bot/chroma_db_with_role"

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    encode_kwargs={"normalize_embeddings": True}
)

vectorstore = Chroma(
    persist_directory=CHROMA_DIR,
    embedding_function=embeddings,
)

collection = vectorstore._collection


In [ ]:
metas = collection.get(include=["metadatas"])["metadatas"]

# 전체 metadata key 집합
keys = set()
for m in metas:
    if m:
        keys.update(m.keys())

print("Metadata keys:", keys)

from collections import defaultdict

topic_segments = defaultdict(list)

for m in metas:
    if not m:
        continue
    topic = m.get("topic")
    if topic:
        topic_segments[topic].append(m)

# 예시: be동사 토픽 상위 5개 출력
for i, seg in enumerate(topic_segments["be동사"][:5]):
    print(f"\nSegment {i}")
    print(json.dumps(seg, ensure_ascii=False, indent=2))


Metadata keys: {'grammar_type', 'topic', 'role', 'keywords', 'start_time', 'video_filename', 'end_time', 'video_url', 'summary'}

Segment 0
{
  "video_url": "https://www.youtube.com/watch?v=V8e_cwY7VTs",
  "topic": "be동사",
  "keywords": "be동사, 학습",
  "role": "definition",
  "grammar_type": "be동사",
  "summary": "be동사 주제 소개",
  "video_filename": "20_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#10 | 바쁘다는 busy가 아니다! | 기초 영어 회화.json",
  "start_time": 10.6,
  "end_time": 14.2
}


In [5]:
import json
import glob
import os
import unicodedata
import re
from openai import OpenAI
import difflib
from dotenv import load_dotenv
from langchain_core.documents import Document


load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

# URL 매핑
video_url_map = {
    "01_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #29 | 과거에 있었던 일 설명할 때  | 기초 영어 회화.json": "https://www.youtube.com/watch?v=R_-pgaQYaYQ",
    "02_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #28 | 마법과 같은 that | 기초 영어 회화.json": "https://www.youtube.com/watch?v=008886a-lQI",
    "03_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 ##27 | 수동태를 that과 연결하기! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=r3qBF9dMz10",
    "04_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #26 | 수동태 핵심 파악! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=bGl_7acUnNk",
    "05_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #25 | have를 깊이, 자연스럽게 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=jOb8mznvX48",
    "06_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #24 | 현재완료 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=RgNbTRRt78Y",
    "07_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #23 | that 뒤에 조동사 쓰기  | 기초 영어 회화.json": "https://www.youtube.com/watch?v=OaTujaboBf8",
    "08_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #22 | that 뒤에 진행형을 가지고 만들어 보자  | 기초 영어 회화.json": "https://www.youtube.com/watch?v=NXbGg9nxpdk",
    "09_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #21 | 또 다른 that | 기초 영어 회화.json": "https://www.youtube.com/watch?v=k3-666q27Ps",
    "10_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #20 | that으로 문장을 길게! (3) | 기초 영어 회화.json": "https://www.youtube.com/watch?v=JMfB_2pfqCA",
    "11_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #19 | that 으로 문장 길게 만들기(2) | 기초 영어 회화.json": "https://www.youtube.com/watch?v=gHrI6qhbziI",
    "12_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#18 | that 으로 문장 길게 만들기(1) | 기초 영어 회화.json": "https://www.youtube.com/watch?v=nHbEN7KEmmE",
    "13_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #17 | can을 be able to로 바꾸기! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=CGN1TdvhkvY",
    "14_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #16 | 동명사로 주어 길게 만들기 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=jzJzdoBdeAc",
    "15_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #15 | 목적을 나타내는 to 동사원형 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=7CvXgPmdD9s",
    "16_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #14 | 미래와 과거에도 진행형을 쓴다! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=7ot7hY8wm4Q",
    "17_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #13 | 현재 진행형의 다양한 쓰임.json": "https://www.youtube.com/watch?v=j87QB9EZZrY",
    "18_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #12 | '~에 있다'를 뜻하는 be 동사 연습 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=h_Yv5bX8p8k",
    "19_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#11 | 다양한 be 동사 형태 연습 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=WS5hLZV7Lb4",
    "20_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#10 | 바쁘다는 busy가 아니다! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=V8e_cwY7VTs",
    "21_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#9 | if로 문장 길게 만들기 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=hh9pAAS-gho",
    "22_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#8 | '~하기를' to 동사원형 연습 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=TK1HL_27g6U",
    "23_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #5  | 의문문 길게 만들기 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=EXKS6rZHbbA",
    "24_NEW 이시원의 기초 영어 회화 강의한 달 만에 영어로 말문 트기 #4 | 과거 시제 마스터 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=G_SNroMhJTQ",
    "25_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#3 | And 로 문장을 길게!  | 기초 영어 회화.json": "https://www.youtube.com/watch?v=VJeidy58uJQ",
    "26_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#1 | 영어는 단어의 연결 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=oLIpoVoDgTo",
    "27_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#2 | 미래를 나타내는 will | 기초 영어 회화.json": "https://www.youtube.com/watch?v=KQbWy6j_TFA",
    "28_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #30 | 많이 쓰는 동사 put / get / take | 기초 영어 회화.json": "https://www.youtube.com/watch?v=1dIALFMvJlA",
    "29_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#6 | 의문문 질문에 답하기 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=a_eNZ4ZVwxc",
    "30_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #7 | 가능과 허락의 can | 기초 영어 회화.json": "https://www.youtube.com/watch?v=kYx8f4U4-jo"
}

def normalize_filename(name):
    name = unicodedata.normalize("NFC", name)
    name = name.replace("｜", "|").replace("＃", "#")
    name = re.sub(r"\s+", " ", name.strip())
    return name


def get_video_url(video_filename):
    keys = list(video_url_map.keys())
    match = difflib.get_close_matches(video_filename, keys, n=1, cutoff=0.6)
    if match:
        return video_url_map[match[0]]
    return "URL_없음"

# LLM으로 텍스트 강화
client = OpenAI()

def enrich_with_llm(raw_text):
    """STT 텍스트를 LLM으로 정제 + 메타데이터 추출"""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": """
                당신은 영어 교육 전문가입니다.
                이시원 선생님의 영어 강의 전사본을 분석하여 다음을 추출하세요:

                1. cleaned_text: 불필요한 말(음, 자, 그러니까 등) 제거하고 핵심만 정리
                2. topic: 주요 문법 주제 (예: "과거완료", "be동사", "동명사")
                3. grammar_type: 문법 분류 (예: "시제", "품사", "문장구조")
                4. difficulty: "초급", "중급", "고급" 중 하나
                5. keywords: 핵심 키워드 배열 (최대 5개)

                JSON 형식으로 반환하세요.
                """},
            {"role": "user", "content": f"다음 강의 내용을 분석하세요:\n\n{raw_text}"}
        ],
        response_format={"type": "json_object"},
        temperature=0
    )
    
    return json.loads(response.choices[0].message.content)

# Documents 생성
json_files = sorted(glob.glob("./youtube_playlist/*.json"))
all_documents = []

print(f"📄 Document 생성 중... ({len(json_files)}개 파일)\n")

for json_file in json_files:
    video_filename = os.path.basename(json_file)
    video_url = get_video_url(video_filename)
    
    print(f"처리 중: {video_filename}")
    
    with open(json_file, "r", encoding="utf-8") as f:
        result = json.load(f)
    
    segments = result["segments"]
    
    # 각 segment를 LLM으로 강화
    for seg in segments:
        raw_text = seg["text"].strip()
        
        # 너무 짧으면 스킵
        if len(raw_text) < 20:
            continue
        
        # LLM으로 강화
        try:
            enriched = enrich_with_llm(raw_text)
        
            # keywords 리스트를 문자열로 변환
            keywords_str = ", ".join(enriched.get("keywords", []))
            
            doc = Document(
                page_content=enriched.get("cleaned_text", raw_text),
                metadata={
                    "video_url": video_url,
                    "video_filename": video_filename,
                    "start_time": seg["start"],
                    "end_time": seg["end"],
                    "topic": enriched.get("topic", ""),
                    "grammar_type": enriched.get("grammar_type", ""),
                    "difficulty": enriched.get("difficulty", ""),
                    "keywords": keywords_str,  # ✅ 문자열로 변환!
                    "raw_text": raw_text,
                }
            )
            all_documents.append(doc)
            
        except Exception as e:
            print(f"⚠️ LLM 처리 실패: {e}")
            # 실패하면 원본 그대로 저장
            doc = Document(
                page_content=raw_text,
                metadata={
                    "video_url": video_url,
                    "video_filename": video_filename,
                    "start_time": seg["start"],
                    "end_time": seg["end"],
                    "raw_text": raw_text,
                }
            )
            all_documents.append(doc)

print(f"\n✅ Document 생성 완료: {len(all_documents)}개")

📄 Document 생성 중... (30개 파일)

처리 중: 01_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #29 ｜ 과거에 있었던 일 설명할 때  ｜ 기초 영어 회화.json
처리 중: 02_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #28 ｜ 마법과 같은 that ｜ 기초 영어 회화.json
처리 중: 03_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 ##27 ｜ 수동태를 that과 연결하기! ｜ 기초 영어 회화.json
처리 중: 04_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #26 ｜ 수동태 핵심 파악! ｜ 기초 영어 회화.json
처리 중: 05_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #25 ｜ have를 깊이, 자연스럽게 ｜ 기초 영어 회화.json
처리 중: 06_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #24 ｜ 현재완료 ｜ 기초 영어 회화.json
처리 중: 07_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #23 ｜ that 뒤에 조동사 쓰기  ｜ 기초 영어 회화.json
처리 중: 08_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #22 ｜ that 뒤에 진행형을 가지고 만들어 보자  ｜ 기초 영어 회화.json
처리 중: 09_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #21 ｜ 또 다른 that ｜ 기초 영어 회화.json
처리 중: 10_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #20 ｜ that으로 문장을 길게! (3) ｜ 기초 영어 회화.json
처리 중: 11_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #19 ｜ that 으로 문장 길게 만들기(2) ｜ 기초 영어 회화.json
처리 중: 12_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기#18 ｜ that 으로 문장 길게 만들기(1) ｜ 기초 영어 회화.json
처리 중: 13_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #17 ｜ can을

In [6]:
all_documents

[Document(metadata={'video_url': 'https://www.youtube.com/watch?v=R_-pgaQYaYQ', 'video_filename': '01_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #29 ｜ 과거에 있었던 일 설명할 때\xa0 ｜ 기초 영어 회화.json', 'start_time': 6.0, 'end_time': 13.0, 'topic': '과거시제', 'grammar_type': '시제', 'difficulty': '초급', 'keywords': '과거, 설명, 친구, 일, 시제', 'raw_text': '이번 시간에는 우리가 과거에 있었던 일을 친구를 설명을 할 거예요.'}, page_content='이번 시간에는 과거에 있었던 일을 친구에게 설명하는 방법을 배웁니다.'),
 Document(metadata={'video_url': 'https://www.youtube.com/watch?v=R_-pgaQYaYQ', 'video_filename': '01_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #29 ｜ 과거에 있었던 일 설명할 때\xa0 ｜ 기초 영어 회화.json', 'start_time': 13.0, 'end_time': 17.0, 'topic': '과거 시제', 'grammar_type': '시제', 'difficulty': '초급', 'keywords': '과거, 종로, 어제, 설명, 친구', 'raw_text': '친구에게 설명을 할 건데 나는 어젯게 종로에 갔어 그러면 어떻게 하면 되죠?'}, page_content="친구에게 설명할 때 '나는 어제 종로에 갔어'라고 말합니다."),
 Document(metadata={'video_url': 'https://www.youtube.com/watch?v=R_-pgaQYaYQ', 'video_filename': '01_NEW 이시원 강의 ｜ 한 달 만에 영어로 말문 트기 #29 ｜ 과거에 있었던 일 설명할 때\xa0 ｜ 기초 영어 회

In [7]:
# 임베딩 모델
import shutil
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

#if os.path.exists("./lecture_db"):
#    shutil.rmtree("./lecture_db")
#    print("✅ 기존 DB 삭제 완료")
    
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large"   #문장임베딩 전용모델
)

# 벡터스토어 생성 (한 방에!)
vectorstore = Chroma.from_documents(
    documents=all_documents,
    persist_directory="./chroma_db",
    embedding=embeddings,
    collection_name="lectures"
)

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 벡터 DB 생성
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

vectorstore = Chroma.from_documents(
    documents=all_documents,
    embedding=embeddings,
    persist_directory="./chroma_db",
    collection_name="english_grammar"
)

# 2. Retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. RAG Chain (새로운 방식)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

template = """
당신은 이시원 선생님의 영어 강의 도우미입니다.
제공된 강의 내용을 기반으로 답변하세요.

답변 시:
1. 명확한 설명
2. 예문 포함
3. 해당 강의 영상 링크 제공

컨텍스트:
{context}

질문: {question}

답변:
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    """Document 리스트를 문자열로 변환"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        video_url = doc.metadata.get('video_url', '')
        start_time = doc.metadata.get('start_time', 0)
        topic = doc.metadata.get('topic', '')
        
        timestamp_url = f"{video_url}&t={int(start_time)}s" if video_url != "URL_없음" else ""
        
        formatted.append(f"""
[{i}] 주제: {topic}
링크: {timestamp_url}
내용: {doc.page_content}
""")
    return "\n".join(formatted)

# RAG Chain 구성
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 테스트
def ask(question):
    print(f"❓ 질문: {question}\n")
    
    # 검색된 문서 확인
    docs = retriever.invoke(question)
    
    # 답변 생성
    answer = rag_chain.invoke(question)
    
    print(f"💬 답변:\n{answer}\n")
    
    print("📚 참고 강의:")
    for i, doc in enumerate(docs[:3], 1):
        video_url = doc.metadata.get('video_url', '')
        start_time = doc.metadata.get('start_time', 0)
        topic = doc.metadata.get('topic', '')
        
        timestamp_url = f"{video_url}&t={int(start_time)}s" if video_url != "URL_없음" else ""
        
        print(f"{i}. [{topic}] {timestamp_url}")
        print(f"   {doc.page_content[:100]}...\n")

# 테스트
ask("동명사가 뭐예요?")

❓ 질문: 동명사가 뭐예요?

💬 답변:
동명사는 영어에서 'verb + -ing' 형태로 사용되는 단어로, 명사처럼 기능하는 동사입니다. 즉, 동명사는 동사의 성질을 가지면서도 문장에서 주어, 목적어, 보어 등으로 사용될 수 있습니다.

예를 들어, "Swimming is fun."에서 "Swimming"은 주어로 사용된 동명사입니다. 이 문장은 "수영하는 것은 재미있다."라는 의미를 가지고 있습니다.

또 다른 예로, "I enjoy reading."에서 "reading"은 목적어로 사용된 동명사입니다. 이 문장은 "나는 읽는 것을 즐긴다."라는 의미입니다.

더 자세한 설명과 예시는 아래의 강의 영상을 통해 확인하실 수 있습니다: [동명사 강의 영상](https://www.youtube.com/watch?v=jzJzdoBdeAc&t=6s)

📚 참고 강의:
1. [동명사] https://www.youtube.com/watch?v=jzJzdoBdeAc&t=6s
   동명사에 대해 설명합니다....

2. [의문문] https://www.youtube.com/watch?v=G_SNroMhJTQ&t=98s
   동대문에 가서 어떻게 하면 될까요?...

3. [동사 위치] https://www.youtube.com/watch?v=NXbGg9nxpdk&t=11s
   됐 다음에 바로 동사가 오는 게 아니고...



In [15]:
# check_preposition.py
import json

with open('/data/edutem/sooine/rag_bot/knowledge_graph.json', 'r', encoding='utf-8') as f:
    kg = json.load(f)

# "전치사" 관련 주제가 main_topic으로 있나?
if "전치사" in kg:
    print("❌ 문제: '전치사'가 main_topic으로 잘못 분류됨")
    print(kg["전치사"])
else:
    print("✅ 괜찮음: '전치사'는 sub_topic으로만 존재")

# "의문문 만들기"에서 전치사 sub_topic 확인
if "의문문 만들기" in kg:
    for sub_id, sub_data in kg["의문문 만들기"]["sub_topics"].items():
        if "전치사" in sub_data["title"]:
            print(f"\n✅ 발견: {sub_data['title']}")
            print(f"   영상: {sub_data['video_segments'][0]['video_title']}")
            print(f"   시간: {sub_data['video_segments'][0]['start_time']}초")

✅ 괜찮음: '전치사'는 sub_topic으로만 존재
